In [ ]:
# !pip install python==3.9
!pip install torch
!pip install deepparse

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import random
# import pytorch
from deepparse.parser import AddressParser
import deepparse


In [ ]:
import os
import shutil

bool_new_model = True
dirpath = os.path.join('checkpoints')

if os.path.exists(dirpath) and os.path.isdir(dirpath) and bool_new_model:
    shutil.rmtree(dirpath)

In [ ]:
def generate_rand_garbage(list_bloc):
    list_garbage = ['',' ','École','Bibliothèque','Musée','Bureau de poste','Banque','Hôtel','Restaurant','Café','Cinéma','Théâtre',
                    'Stade','Gymnase','Parc aquatique','Magasin de vêtements','Pharmacie','Épicerie','Salon de coiffure','Station service',
                    'Garage','Boutique de jouets','Galerie d art','Studio de danse','Salle de concert','Centre commercial','Boulangerie',
                    'Université','Poste de police','Caserne de pompiers','Église','Mosquée','Synagogue','Temple','Hôpital','Clinique','Cabinet médical',
                    'Pharmacie','Maison de retraite','Centre de réadaptation','Salle de conférence','Salle d exposition',
                    'School','Bibliotheek','Museum','Postkantoor','Bank','Hotel','Restaurant','Café','Bioscoop','Theater','Stadion',
                    'Sportschool','Waterpark','Kledingwinkel','Apotheek','Kruidenierswinkel','Kapsalon','Tankstation','Garage','Speelgoedwinkel',
                    'Kunstgalerie','Dansstudio','Concertzaal','Winkelcentrum','Bakkerij','Universiteit','Politiebureau','Brandweerkazerne',
                    'Kerk','Moskee','Synagoge','Tempel','Ziekenhuis','Kliniek','Huisartsenpraktijk','Apotheek','Verpleeghuis','Revalidatiecentrum',
                    'Conferentieruimte','Tentoonstellingsruimte']
    garbage = random.choice(list_garbage)
    list_extension = ['', ' de ' + list_bloc[4], ' van ' + list_bloc[4]]
    extension = random.choice(list_extension)
    if garbage == '': 
        return ''
    else : 
        return garbage + extension
    
def generate_rand_box(list_bloc):
    list_box = ['','box','boite','BOITE','bte', 'b', 'bt']
    str_box = random.choice(list_box)
    if list_bloc[2] == '' or  list_bloc[2] is None: 
        return ''
    else : 
        return str_box + ' ' + list_bloc[2] # TODO AJOUTER RANDOMNESS SUR SPACE ENTRE BOITE ET NUM DE BOITE

def del_symbols(list_bloc):
    result = []
    symbols = "!@#$%^&*()_-+={}[]|\:;'<>?,./\""
    for item in list_bloc:
        if item is not None:
            new_item = ''.join([c if c.isalnum() or c.isspace() else ' ' for c in item])
            result.append(new_item)
        else :
            result.append(None)
    if (len(list_bloc) != len(result)):
        raise str(len(list_bloc)) + ' != ' + str(len(result))
    return result 

def generate_rand_presence(list_bloc):
    N = len(list_bloc)
    random_list = [random.choices([0,1], [0.2,0.8])[0] for _ in range(N)]
    return random_list

def generate_rand_order(list_bloc):
    N = len(list_bloc)
    unique_list = list(range(1, N + 1))
    random.shuffle(unique_list)
    return unique_list

def add_typo_error(list_bloc, list_order):
    result = []
    error_type = random.choice(['nothing','add', 'delete'])
    if len(list_bloc) != len(list_order):
        print(list_bloc)
        raise str(list_bloc)
    for i in range(len(list_bloc)):
        string = list_bloc[i]
        order = list_order[i]
        if string is None:
            pass
        elif error_type == 'nothing' or order=='PostalCode' or len(string)<1: 
            pass
        elif error_type == 'add':
            error_index = random.randint(0, len(string))
            error_char = random.choice('abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ')
            string = string[:error_index] + error_char + string[error_index:]
        elif error_type == 'delete' and len(string)>1:
            if len(string) > 0:
                error_index = random.randint(0, len(string) - 1)
                string = string[:error_index] + string[error_index+1:]
        result.append(string)
        
    if (len(list_bloc) != len(result)):
        raise str(list_bloc) + ' != ' + str(result)
    return result

def generate_elem_category(list_bloc, list_order = ['StreetNumber','StreetName','Unit','PostalCode','Municipality','GeneralDelivery']):
    list_elem_category = []
    if len(list_bloc) != len(list_order):
        print('il n\'y a pas autant de blocs que de categories ' + str(len(list_bloc)) + ' != ' + str(len(list_order)))
        raise 'il n\'y a pas autant de blocs que de categories ' + str(len(list_bloc)) + ' != ' + str(len(list_order))
    for i in range(len(list_bloc)):
        bloc = list_bloc[i]
        order = list_order[i]
        if bloc is not None:
            len_bloc = len(bloc.split())
            elem_category = [order] * len_bloc
            list_elem_category.append(elem_category)
        else : 
            list_elem_category.append([order])
    return list_elem_category

def change_list_order(lst, order):
    if len(lst) != len(order):
        raise ValueError("List and order must have the same length.")
    new_lst = [None] * len(lst)
    for i, value in enumerate(lst):
        new_index = order[i] - 1  
        new_lst[new_index] = value
    return new_lst

def address_augmentation(list_bloc, list_order):
    
    list_bloc[-1] = generate_rand_garbage(list_bloc)
    list_bloc[2] = generate_rand_box(list_bloc)
    # Pour la rue, remplacer des mots par une de leurs possibles abréviations
    # importer / creer json de toutes les abréviations possibles
    # print(list_bloc)
    list_bloc = del_symbols(list_bloc.copy())
    # print(list_bloc)
    list_bloc = add_typo_error(list_bloc.copy(), list_order)
    # print(list_bloc)
    list_elem_cat = generate_elem_category(list_bloc)
    rand_presence = generate_rand_presence(list_bloc)
    rand_order = generate_rand_order(list_bloc)
    
    list_bloc_ordered = change_list_order(list_bloc,rand_order)
    list_elem_cat_ordered = change_list_order(list_elem_cat,rand_order)
    rand_presence_ordered = change_list_order(rand_presence,rand_order)
    
    list_final_category = []
    string_final = ''
    for i in range(len(rand_presence_ordered)):
        if rand_presence_ordered[i] == 1:
            if list_bloc_ordered[i] is None:
                list_final_category.append(list_elem_cat_ordered[i])
            elif len(list_bloc_ordered[i].strip()) != 0: 
                list_final_category.append(list_elem_cat_ordered[i])
            if list_bloc_ordered[i] is not None and i==0:
                string_final = list_bloc_ordered[i]
            elif list_bloc_ordered[i] is not None:
                string_final = string_final + ' ' + list_bloc_ordered[i]
    string_final = string_final.strip()
    words = string_final.split()
    string_final = ' '.join(words)
    flattened_list_final_category = [element for sublist in list_final_category for element in sublist]
    
    return string_final, flattened_list_final_category

In [ ]:
!pip install pytest
!pip install mock

In [ ]:
from unittest import mock

# ['StreetName','StreetNumber','Unit','PostalCode','Municipality','GeneralDelivery']
list_bloc1 = ['86','Avenue des Camélias','','1150','Woluwé-Saint-Pierre','']
list_bloc2 = ['22','Belgielaan','','1800','Vilvoorde','']
list_bloc3 = ['115','Rue inventée','4','1200','Woluwé-Saint-Lambert','Boutique de jouets']
list_bloc4 = ['8','Avenue de Boileau','5','1160','Auderghem','Magasin de vêtements de Auderghem']
list_bloc5 = ['8','Avenue de Boileau','5','1160','Auderghem','']
list_bloc6 = ['115','Rue inventée','4','1200','Woluwé-Saint-Lambert','']

# test part
def test_generate_rand_garbage():
    with mock.patch('random.choice') as mock_choice:
        mock_choice.side_effect = ['',' de ' + list_bloc1[4]]
        g = generate_rand_garbage(list_bloc1)
        assert g == '', g
    with mock.patch('random.choice') as mock_choice:
        mock_choice.side_effect = ['Bibliothèque', ' de ' + list_bloc2[4]]
        g = generate_rand_garbage(list_bloc2)
        assert g == 'Bibliothèque de Vilvoorde', g

def test_del_symbols():
    g1 = del_symbols(list_bloc1)
    assert g1 == ['86','Avenue des Camélias','','1150','Woluwé Saint Pierre',''], g1
    g2 = del_symbols(list_bloc6)
    assert g2 == ['115','Rue inventée','4','1200','Woluwé Saint Lambert',''], g2

def test_generate_rand_presence():
    g1 = generate_rand_presence(list_bloc1)
    assert sum(g1) <= 6, g1
    g2 = generate_rand_presence(list_bloc2)
    assert sum(g2) >= 0, g2

def test_generate_rand_order():
    g = generate_rand_order(list_bloc3)
    g_sorted = sorted(g)
    assert g_sorted == [1,2,3,4,5,6], g

'''
def test_add_typo_error():
    with mock.patch('random.randint') as mock_index, \
         mock.patch('random.choice') as mock_choice:  
        mock_index.return_value = 5
        mock_choice.return_value = 'delete'
        assert add_typo_error(list_bloc) == 
'''

# ['8','Avenue de Boileau','5','1160','Auderghem','Magasin de vêtements de Auderghem']
def test_generate_elem_category():
    g = generate_elem_category(list_bloc4, list_order = ['StreetNumber','StreetName','Unit','PostalCode','Municipality','GeneralDelivery'])
    assert g == \
        [['StreetNumber'],['StreetName','StreetName','StreetName'],['Unit'],['PostalCode'],['Municipality'],['GeneralDelivery','GeneralDelivery','GeneralDelivery','GeneralDelivery','GeneralDelivery']], g

"""
def test_change_list_order():
    assert change_list_order(lst, order) == 

def test_address_augmentation():
    assert address_augmentation(list_bloc) == 
"""

test_generate_rand_garbage()
test_del_symbols()
test_generate_rand_presence()
test_generate_rand_order()
test_generate_elem_category()

In [ ]:
list_add = [list_bloc1,list_bloc2,list_bloc3,list_bloc4,list_bloc5,list_bloc6]
list_order = ['StreetNumber','StreetName','Unit','PostalCode','Municipality','GeneralDelivery']
print(list_add)
for s in list_add:    
    # print(s)
    # print(type(s))
    # print(len(s))
    print(address_augmentation(s,list_order))

Dataset Preprocessing part 1: join muni and best

In [ ]:
df_muni = pd.read_csv('/kaggle/input/municipalities/muni.csv')
df_muni = df_muni.astype(str)
df_muni = df_muni.replace('nan', None)
df_muni.columns = 'muni_' + df_muni.columns
df_muni['muni_validity_from'] = pd.to_datetime(df_muni['muni_validity_from'])
df_muni = df_muni.sort_values('muni_validity_from', ascending=False)
df_muni = df_muni.drop_duplicates(subset='muni_muni_code',keep='first').reset_index()
print(df_muni)

# df_muni = df_muni[['muni_muni_code','muni_name_fr','muni_name_nl', 'muni_name_de']]

df_muni_fr= df_muni[['muni_muni_code','muni_name_fr']]
df_muni_fr.columns = ['muni_code','muni_name']

df_muni_nl= df_muni[['muni_muni_code','muni_name_nl']]
df_muni_nl.columns = ['muni_code','muni_name']

df_muni_de= df_muni[['muni_muni_code','muni_name_de']]
df_muni_de.columns = ['muni_code','muni_name']

del df_muni
df_muni = df_muni_fr
del df_muni_fr
df_muni = pd.concat([df_muni, df_muni_nl], axis=0)
del df_muni_nl
df_muni = pd.concat([df_muni, df_muni_de], axis=0)
del df_muni_de

df_muni.dropna(subset = ['muni_name'], inplace=True)
df_muni.drop_duplicates(subset = ['muni_name'], inplace=True)

print(df_muni)

In [ ]:
df_best = pd.read_csv('/kaggle/input/best-address/best.csv')
df_best = df_best.astype(str)
df_best = df_best.replace('nan', None)
df_best.columns = 'best_' + df_best.columns

df_best = df_best[['best_streetname_fr', 'best_streetname_nl',
                  'best_streetname_de', 'best_house_number', 'best_box_number', 'best_muni_code', 'best_poco_code']]

df_best_fr = df_best[['best_streetname_fr','best_house_number', 'best_box_number', 'best_muni_code', 'best_poco_code']]
df_best_fr.columns = ['best_streetname','best_house_number', 'best_box_number', 'best_muni_code', 'best_poco_code']
df_best_nl = df_best[['best_streetname_nl','best_house_number', 'best_box_number', 'best_muni_code', 'best_poco_code']]
df_best_nl.columns = ['best_streetname','best_house_number', 'best_box_number', 'best_muni_code', 'best_poco_code']
df_best_de = df_best[['best_streetname_de','best_house_number', 'best_box_number', 'best_muni_code', 'best_poco_code']]
df_best_de.columns = ['best_streetname','best_house_number', 'best_box_number', 'best_muni_code', 'best_poco_code']

df_best = df_best_fr
del df_best_fr
df_best = pd.concat([df_best, df_best_nl], axis=0)
del df_best_nl
df_best = pd.concat([df_best, df_best_de], axis=0)
del df_best_de

df_best.dropna(subset = ['best_streetname'], inplace=True)
df_best.drop_duplicates(subset = ['best_streetname','best_house_number','best_box_number','best_muni_code','best_poco_code'], inplace=True)

print(df_best)


In [ ]:
address = pd.merge(df_best,df_muni,left_on=['best_muni_code'],right_on=['muni_code'])
del df_best
del df_muni
# ['StreetNumber','StreetName','Unit','PostalCode','Municipality','GeneralDelivery']
address = address[['best_house_number',\
                   'best_streetname',\
                   'best_box_number',\
                   'best_poco_code',\
                   'muni_name']]
address['GeneralDelivery'] = ''
print(address)

In [ ]:
print(address.dtypes)
test = address.iloc[0:2].values.tolist()
print(test)
# print(np.isnan(test[0][2]))

In [ ]:
address_parser = AddressParser(model_type="bpemb", device=0)
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Dataset Preprocessing part 2: Augmentation

In [ ]:
list_order = ['StreetNumber','StreetName','Unit','PostalCode','Municipality','GeneralDelivery']

print('debut part 1')
list_address = address.values.tolist()
print(len(list_address))
print('step1 fini')
# prinst_address)
list_2_container = [] 

for add in list_address:
    inp, out = address_augmentation(add,list_order)
    if inp == '' and out == []:
        pass
    elif (inp != '' and out == []) or (inp == '' and out != []):
        raise 'mauvaise correspondance entre inp et out'
    else:
        list_2_container.append((inp, out))
    # print([inp, out])
print('step2 fini')

In [ ]:
print(list_2_container[2]) 

random.shuffle(list_2_container)

print(list_2_container[2]) 

Dataset Preprocessing part 3: list to container

In [ ]:
'''
address_parser.retrain(training_container,
                       train_ratio=0.8,
                       epochs=5,
                       batch_size=8,
                       num_workers=2,
                       callbacks=[lr_scheduler],
                       prediction_tags=tag_dictionary,
                       logging_path=logging_path)
'''
'''
# list_2_containerTEST = [('12 belgielaan bxA',['StreetNumber','StreetName','Unit']),
#                     ('12 belgielaan bxA',['StreetNumber','StreetName','Unit'])]
# container_train = deepparse.dataset_container.ListDatasetContainer(list_2_containerTEST)
random.shuffle(list_2_container)
container_train = deepparse.dataset_container.ListDatasetContainer(list_2_container[:100000])
print(list_2_container[:10])
address_parser.retrain(container_train,
                       train_ratio=0.8,
                       epochs=14,
                       batch_size=20)'''

In [ ]:
random.shuffle(list_2_container)

batch_size = 300000  # Number of rows to process in each iteration

total_rows = len(list_2_container)
num_batches = (total_rows + batch_size - 1) // batch_size  # Calculate the number of batches

N_epoch = address_parser.nb_epoch + 1

for batch_index in range(num_batches):
    start_index = batch_index * batch_size
    end_index = min((batch_index + 1) * batch_size, total_rows)
    
    batch = list_2_container[start_index:end_index]
    
    container_train = deepparse.dataset_container.ListDatasetContainer(batch)

    address_parser.retrain(container_train,
                           train_ratio=0.8,
                           epochs=N_epoch,
                           batch_size=25)
    N_epoch = N_epoch + 1

In [ ]:
# Save model architecture as JSON
model_json = address_parser.to_json()
with open("model_parserV1.json", "w") as json_file:
    json_file.write(model_json)

# Save model weights
address_parser.save_weights("model_parserV1_weights.h5")

In [ ]:
list_order = ['StreetNumber','StreetName','Unit','PostalCode','Municipality','GeneralDelivery']

N_augmentation = 20

for u in range(N_augmentation):
    print('debut part 1')
    list_address = address.values.tolist()
    print(len(list_address))
    print('step1 fini')
    # prinst_address)
    list_2_container = [] 

    for add in list_address:
        inp, out = address_augmentation(add,list_order)
        if inp == '' and out == []:
            pass
        elif (inp != '' and out == []) or (inp == '' and out != []):
            raise 'mauvaise correspondance entre inp et out'
        else:
            list_2_container.append((inp, out))
        # print([inp, out])
    print('step2 fini')
    
    random.shuffle(list_2_container)

    batch_size = 100000  # Number of rows to process in each iteration
    total_rows = len(list_2_container)
    num_batches = (total_rows + batch_size - 1) // batch_size  # Calculate the number of batches

    N_epoch = 15

    for batch_index in range(num_batches):
        start_index = batch_index * batch_size
        end_index = min((batch_index + 1) * batch_size, total_rows)

        batch = list_2_container[start_index:end_index]

        container_train = deepparse.dataset_container.ListDatasetContainer(batch)

        address_parser.retrain(container_train,
                               train_ratio=0.8,
                               epochs=N_epoch,
                               batch_size=25)
        N_epoch = N_epoch + 1